# Training Notebook for F1-tenth team

Only run this notebook directly on Google Colab! Much easier to handle dependencies and way faster in training.

## Install dependencies

Automatically install dependencies here. Please make sure to spin up the GPU T4 instance for this notebook to ensure training speed.

In [ ]:
!uv pip install ultralytics comet_ml

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.3/112.6 GB disk)


## Monitoring

Setup model monitoring with Comet for deep insights into how well the model was trained (optional)

In [ ]:
# import comet_ml

# comet_ml.login(project_name="path-segmentation", workspace="d3lta-v")

import ultralytics
ultralytics.checks()

In [4]:
!pwd

/content


## Data Ingestion

Upload both dataset.zip and data.yaml, then run this box

In [9]:
!unzip /content/dataset.zip

Archive:  /content/dataset.zip
replace dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Training

In [ ]:
from ultralytics import YOLO

def train_yolov11_segmentation(
    data_yaml_path,
    model_name="yolo11n-seg.pt",
    epochs=100,
    imgsz=640,
    batch_size=16,
    device=0,
    project_name="segmentation_training",
    run_name="custom_seg_model",
    pretrained=True,
    resume=False,
    patience=100,
    save_period=-1,
    workers=8,
    lr0=0.01, # Initial learning rate
    optimizer="auto"
):
    """
    Train YOLOv11n-seg model on custom segmentation dataset.

    Args:
        data_yaml_path (str): Path to dataset YAML configuration file
        model_name (str): Model to use (e.g., 'yolo11n-seg.pt', 'yolo11s-seg.pt')
        epochs (int): Number of training epochs
        imgsz (int): Input image size
        batch_size (int): Batch size (use -1 for auto mode)
        device (int/str): Training device (0 for GPU, 'cpu', 'mps' for Apple Silicon)
        project_name (str): Project directory name
        run_name (str): Training run name
        pretrained (bool): Use pretrained weights
        resume (bool): Resume from last checkpoint
        patience (int): Early stopping patience
        save_period (int): Save checkpoint every n epochs (-1 to disable)
        workers (int): Number of data loader workers
        lr0 (float): Initial learning rate
        optimizer (str): Optimizer choice ('SGD', 'Adam', 'AdamW', 'auto')

    Returns:
        results: Training results object
    """

    # Load model
    if resume:
        # Resume from checkpoint
        model = YOLO("path/to/last.pt")
        print("Resuming training from checkpoint...")
    else:
        if pretrained:
            # Load pretrained model (recommended)
            model = YOLO(model_name)
            print(f"Loaded pretrained model: {model_name}")
        else:
            # Build new model from YAML
            yaml_name = model_name.replace(".pt", ".yaml")
            model = YOLO(yaml_name)
            print(f"Built new model from: {yaml_name}")

    # Train the model
    print(f"\nStarting training on {device}...")
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        device=device,
        project=project_name,
        name=run_name,
        exist_ok=True,
        pretrained=pretrained,
        optimizer=optimizer,
        lr0=lr0,
        patience=patience,
        save_period=save_period,
        workers=workers,
        val=True,
        plots=True,
        ### Augmentation settings
        # hsv_h=0.015,
        # hsv_s=0.7,
        # hsv_v=0.4,
        # degrees=0.0,
        # translate=0.1,
        # scale=0.5,
        # shear=0.0,
        # perspective=0.0,
        # flipud=0.0,
        # fliplr=0.5,
        # mosaic=1.0,
        # mixup=0.0,
        # copy_paste=0.0,
        ### Regularization
        # dropout=0.0,
        # weight_decay=0.0005,
        ### Loss weights
        # box=7.5,
        # cls=0.5,
        # mask_ratio=4,
        # overlap_mask=True,
        ### Mixed precision training
        # amp=True,
        # Caching (speeds up training)
        cache=False,  # Set to True or 'ram' to cache images in memory
        resume=resume
    )

    print("\nTraining completed!")
    print(f"Results saved to: {project_name}/{run_name}")

    return results


def validate_model(model_path, data_yaml_path, imgsz=640, device=0):
    """
    Validate trained segmentation model.

    Args:
        model_path (str): Path to trained model weights
        data_yaml_path (str): Path to dataset YAML
        imgsz (int): Image size for validation
        device (int/str): Device for validation

    Returns:
        metrics: Validation metrics
    """
    model = YOLO(model_path)
    metrics = model.val(data=data_yaml_path, imgsz=imgsz, device=device)

    print("\nValidation Results:")
    print(f"Box mAP50-95: {metrics.box.map:.4f}")
    print(f"Box mAP50: {metrics.box.map50:.4f}")
    print(f"Mask mAP50-95: {metrics.seg.map:.4f}")
    print(f"Mask mAP50: {metrics.seg.map50:.4f}")

    return metrics


def run_inference(model_path, source, imgsz=640, conf=0.25, device=0, save=True):
    """
    Run inference on images/video with trained model.

    Args:
        model_path (str): Path to trained model
        source (str): Path to image, video, directory, or webcam (0)
        imgsz (int): Image size
        conf (float): Confidence threshold
        device (int/str): Device for inference
        save (bool): Save results

    Returns:
        results: Inference results
    """
    model = YOLO(model_path)
    results = model.predict(
        source=source,
        imgsz=imgsz,
        conf=conf,
        device=device,
        save=save,
        show=False
    )

    # Access segmentation masks
    for i, result in enumerate(results):
        print(f"\nImage {i}:")
        if result.masks is not None:
            masks = result.masks.data  # Binary masks (num_objects x H x W)
            xy = result.masks.xy  # Polygon format
            xyn = result.masks.xyn  # Normalized polygon format
            print(f"  Detected {len(masks)} objects")
            print(f"  Masks shape: {masks.shape}")

    return results


if __name__ == "__main__":
    # Dataset YAML file path
    # This should point to your data.yaml configuration file
    DATA_YAML = "data_colab.yaml"

    # Configuration parameters
    MODEL = "yolo11n-seg.pt"  # Options: yolo11n-seg.pt, yolo11s-seg.pt, yolo11m-seg.pt
    DEVICE = 0  # 0 for GPU, 'cpu', or 'mps' for Apple Silicon
    IMGSZ = 320 # Same as the training image size

    # Training parameters
    EPOCHS = 100
    BATCH = 16  # Use -1 for auto batch sizing

    # Train the model
    results = train_yolov11_segmentation(
        data_yaml_path=DATA_YAML,
        model_name=MODEL,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch_size=BATCH,
        device=DEVICE,
        project_name="yolo_segmentation",
        run_name="lane_detection_v1",
        pretrained=True,
        patience=50
    )

    # Validate the trained model
    best_model_path = "yolo_segmentation/lane_detection_v1/weights/best.pt"
    metrics = validate_model(best_model_path, DATA_YAML, device=DEVICE)

    # Run inference on test images
    # results = run_inference(
    #     model_path=best_model_path,
    #     source="path/to/test/images",
    #     conf=0.25,
    #     device=DEVICE
    # )


Loaded pretrained model: yolo11n-seg.pt

Starting training on 0...
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=lane_detection_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap

## Download

The following cell zips up the results and allows you to download the results.

In [7]:
!zip -r /content/yolo_segmentation.zip yolo_segmentation/

  adding: yolo_segmentation/ (stored 0%)
  adding: yolo_segmentation/lane_detection_v1/ (stored 0%)
  adding: yolo_segmentation/lane_detection_v1/BoxR_curve.png (deflated 17%)
  adding: yolo_segmentation/lane_detection_v1/train_batch2.jpg (deflated 4%)
  adding: yolo_segmentation/lane_detection_v1/train_batch1532.jpg (deflated 11%)
  adding: yolo_segmentation/lane_detection_v1/MaskR_curve.png (deflated 17%)
  adding: yolo_segmentation/lane_detection_v1/MaskPR_curve.png (deflated 30%)
  adding: yolo_segmentation/lane_detection_v1/MaskF1_curve.png (deflated 16%)
  adding: yolo_segmentation/lane_detection_v1/BoxPR_curve.png (deflated 29%)
  adding: yolo_segmentation/lane_detection_v1/confusion_matrix.png (deflated 39%)
  adding: yolo_segmentation/lane_detection_v1/confusion_matrix_normalized.png (deflated 35%)
  adding: yolo_segmentation/lane_detection_v1/train_batch1.jpg (deflated 5%)
  adding: yolo_segmentation/lane_detection_v1/labels.jpg (deflated 31%)
  adding: yolo_segmentation/lane

In [8]:
!zip -r /content/runs.zip runs/

  adding: runs/ (stored 0%)
  adding: runs/segment/ (stored 0%)
  adding: runs/segment/val/ (stored 0%)
  adding: runs/segment/val/val_batch1_labels.jpg (deflated 11%)
  adding: runs/segment/val/BoxR_curve.png (deflated 17%)
  adding: runs/segment/val/MaskR_curve.png (deflated 17%)
  adding: runs/segment/val/MaskPR_curve.png (deflated 26%)
  adding: runs/segment/val/val_batch1_pred.jpg (deflated 10%)
  adding: runs/segment/val/MaskF1_curve.png (deflated 15%)
  adding: runs/segment/val/BoxPR_curve.png (deflated 27%)
  adding: runs/segment/val/confusion_matrix.png (deflated 38%)
  adding: runs/segment/val/confusion_matrix_normalized.png (deflated 35%)
  adding: runs/segment/val/val_batch0_labels.jpg (deflated 8%)
  adding: runs/segment/val/BoxF1_curve.png (deflated 15%)
  adding: runs/segment/val/BoxP_curve.png (deflated 15%)
  adding: runs/segment/val/val_batch0_pred.jpg (deflated 7%)
  adding: runs/segment/val/MaskP_curve.png (deflated 15%)
